In [44]:
import pandas as pd
import numpy as np
import csv
import os
from datetime import datetime
import matplotlib.pyplot as plt
import random
import seaborn as sns
from tsfresh import extract_relevant_features
from tsfresh.utilities.dataframe_functions import impute

In [45]:
#Activate the first option if whole column width is necessary and the second to make all columns visible
#pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)

## I. Import the transformed datasets

In [46]:
#Get the three reports and store them into dataframes. Check what columns are included in the file
events = pd.read_csv('Det_curve_set2.csv', index_col = 0)
rci = pd.read_csv('rci_aggegated.csv', index_col = 0)
base = pd.read_csv('RCI_Score_base.csv', index_col = 0)
temp = pd.read_csv('WeatherTraining_Set_v1.csv', index_col = 0)
traffic = pd.read_csv('AADT_processed_v1.csv', index_col = 0)
grit_routes = pd.read_csv('grit_routes.csv', index_col = 0)
grit_runs = pd.read_csv('grit_runs.csv', index_col = 0)

print(rci.columns)
print(events.columns)
print(base.columns)
print(temp.columns)
print(traffic.columns)
print(grit_routes.columns)
print(grit_runs.columns)

print(rci.shape)
print(events.shape)
print(base.shape)
print(temp.shape)
print(traffic.shape)
print(grit_routes.shape)
print(grit_runs.shape)


Index(['feature_id', 'FinancialY', 'lane', 'avg_score_lane', 'sub_obs_val_max',
       'sub_obs_val_std'],
      dtype='object')
Index(['feature_id', 'FinancialY', 'sub_obs_val_max', 'sub_obs_val_std',
       'Carriageway Scheme', 'Drainage', 'Drainage CAT 2',
       'Localised Patching', 'Long Term Structural Maintenance',
       'Major Patching', 'Microasphalt', 'Reconstruction', 'Recycling',
       'Super Cat 2', 'Surface Dressing', 'Surface Inlay', 'Surface Overlay',
       'Surface Preservative', 'Thin Surfacing', 'Years_since', 'CCD1', 'CCD3',
       'CCD4', 'CFL', 'CFS1', 'CKD2', 'CKD3', 'CMD3', 'CMUD', 'CPH2', 'CPH3',
       'CPH4', 'CPH7', 'CSH1', 'DCF1', 'DCF2', 'DCF3', 'DPD1', 'DPD2', 'GUL',
       'NZU', 'PDAM', 'RICE', 'SFL', 'SPL', 'SPLE', 'Prev_RCI', 'SURF', 'BEDR',
       'Month', 'hierarchy', 'distance', 'road_type_hw', 'urban_rural',
       'surf_type', 'width', 'sub_obs_val_avg', 'weighted_av', 'perc85'],
      dtype='object')
Index(['feature_id', 'FinancialY', 'sub_

In [47]:
base.drop(columns = ['sub_obs_val_med','sub_obs_val_max', 'sub_obs_val_std'], inplace = True)
events.drop(columns = ['sub_obs_val_max', 'sub_obs_val_std', 'Prev_RCI'], inplace = True)
temp.drop(columns = ['sub_obs_val_med', 'sub_obs_val_max',
       'sub_obs_val_std', 'hierarchy', 'distance', 'road_type_hw',
       'urban_rural', 'surf_type','width', 'SITE_CODE', 'Station',
       'Distance_between_Features','Prev_RCI'], inplace = True)
traffic.drop(columns = ['SITE_CODE', 'SITE_NAME', 'TOWN', 'ROADCLASS',
       'HIERARCHY', 'CUSTOMER', 'AREA_NAME', 'DISTANCE', 'ROAD_TYPE',
       'Confidence', 'AADT_Source_Comb', 'AADT_Source_Type',
       'ModifiedDate', 'Checked', 'SurveyID'], inplace = True)
grit_runs.drop(columns = ['Salt30gr', 'Salt15gr', 'Salt8gr'], inplace = True)
traffic.rename(columns = {'FEATURE_ID':'feature_id' }, inplace = True)

### I. Merge the factors with the base

In [48]:
#Add the RCI factors to the base set:
initial = base.merge(rci,'left',on=['feature_id', 'FinancialY'],left_index = True)
initial.reset_index(drop= True, inplace = True)

In [49]:
#Add the events set to the base:
ev_set = initial.merge(events,'left',on=['feature_id', 'FinancialY'],left_index = True)
ev_set.reset_index(drop= True, inplace = True)

In [50]:
#Separate left and right lane sets to add the previous RCI
left = ev_set.loc[ev_set['lane']=='LEFT']
right = ev_set.loc[ev_set['lane']=='RIGHT']
left.reset_index(drop= True,inplace = True)
right.reset_index(drop= True,inplace = True)

In [51]:
print(left.shape)
print(right.shape)
print(ev_set.shape)

(17349, 60)
(9417, 60)
(26766, 60)


In [52]:
#Add the previous RCI score to the training set ***Separate per lane as well
left['Prev_RCI']= np.nan

for i in range(0,len(left)):
    feat = left.iloc[i,0]
    year = left.iloc[i,1]
    
    print(i)
    
    for t in range(0,10):
        year-=1
        ind = left.loc[(left["feature_id"] == feat) & 
                                (left["FinancialY"] == year)].index

        if len(ind) > 0:
            left.iloc[i,-1] = left.iloc[ind,3].sum()
            break
        
        else:
            left.iloc[i,-1] = left.iloc[i,3].sum()          
            continue

C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059


3498
3499
3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697


5136
5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335


...

11407
11408
11409
11410
11411
11412
11413
11414
11415
11416
11417
11418
11419
11420
11421
11422
11423
11424
11425
11426
11427
11428
11429
11430
11431
11432
11433
11434
11435
11436
11437
11438
11439
11440
11441
11442
11443
11444
11445
11446
11447
11448
11449
11450
11451
11452
11453
11454
11455
11456
11457
11458
11459
11460
11461
11462
11463
11464
11465
11466
11467
11468
11469
11470
11471
11472
11473
11474
11475
11476
11477
11478
11479
11480
11481
11482
11483
11484
11485
11486
11487
11488
11489
11490
11491
11492
11493
11494
11495
11496
11497
11498
11499
11500
11501
11502
11503
11504
11505
11506
11507
11508
11509
11510
11511
11512
11513
11514
11515
11516
11517
11518
11519
11520
11521
11522
11523
11524
11525
11526
11527
11528
11529
11530
11531
11532
11533
11534
11535
11536
11537
11538
11539
11540
11541
11542
11543
11544
11545
11546
11547
11548
11549
11550
11551
11552
11553
11554
11555
11556
11557
11558
11559
11560
11561
11562
11563
11564
11565
11566
11567
11568
11569
11570
11571
11572
1157

12772
12773
12774
12775
12776
12777
12778
12779
12780
12781
12782
12783
12784
12785
12786
12787
12788
12789
12790
12791
12792
12793
12794
12795
12796
12797
12798
12799
12800
12801
12802
12803
12804
12805
12806
12807
12808
12809
12810
12811
12812
12813
12814
12815
12816
12817
12818
12819
12820
12821
12822
12823
12824
12825
12826
12827
12828
12829
12830
12831
12832
12833
12834
12835
12836
12837
12838
12839
12840
12841
12842
12843
12844
12845
12846
12847
12848
12849
12850
12851
12852
12853
12854
12855
12856
12857
12858
12859
12860
12861
12862
12863
12864
12865
12866
12867
12868
12869
12870
12871
12872
12873
12874
12875
12876
12877
12878
12879
12880
12881
12882
12883
12884
12885
12886
12887
12888
12889
12890
12891
12892
12893
12894
12895
12896
12897
12898
12899
12900
12901
12902
12903
12904
12905
12906
12907
12908
12909
12910
12911
12912
12913
12914
12915
12916
12917
12918
12919
12920
12921
12922
12923
12924
12925
12926
12927
12928
12929
12930
12931
12932
12933
12934
12935
12936
12937
1293

14137
14138
14139
14140
14141
14142
14143
14144
14145
14146
14147
14148
14149
14150
14151
14152
14153
14154
14155
14156
14157
14158
14159
14160
14161
14162
14163
14164
14165
14166
14167
14168
14169
14170
14171
14172
14173
14174
14175
14176
14177
14178
14179
14180
14181
14182
14183
14184
14185
14186
14187
14188
14189
14190
14191
14192
14193
14194
14195
14196
14197
14198
14199
14200
14201
14202
14203
14204
14205
14206
14207
14208
14209
14210
14211
14212
14213
14214
14215
14216
14217
14218
14219
14220
14221
14222
14223
14224
14225
14226
14227
14228
14229
14230
14231
14232
14233
14234
14235
14236
14237
14238
14239
14240
14241
14242
14243
14244
14245
14246
14247
14248
14249
14250
14251
14252
14253
14254
14255
14256
14257
14258
14259
14260
14261
14262
14263
14264
14265
14266
14267
14268
14269
14270
14271
14272
14273
14274
14275
14276
14277
14278
14279
14280
14281
14282
14283
14284
14285
14286
14287
14288
14289
14290
14291
14292
14293
14294
14295
14296
14297
14298
14299
14300
14301
14302
1430

15502
15503
15504
15505
15506
15507
15508
15509
15510
15511
15512
15513
15514
15515
15516
15517
15518
15519
15520
15521
15522
15523
15524
15525
15526
15527
15528
15529
15530
15531
15532
15533
15534
15535
15536
15537
15538
15539
15540
15541
15542
15543
15544
15545
15546
15547
15548
15549
15550
15551
15552
15553
15554
15555
15556
15557
15558
15559
15560
15561
15562
15563
15564
15565
15566
15567
15568
15569
15570
15571
15572
15573
15574
15575
15576
15577
15578
15579
15580
15581
15582
15583
15584
15585
15586
15587
15588
15589
15590
15591
15592
15593
15594
15595
15596
15597
15598
15599
15600
15601
15602
15603
15604
15605
15606
15607
15608
15609
15610
15611
15612
15613
15614
15615
15616
15617
15618
15619
15620
15621
15622
15623
15624
15625
15626
15627
15628
15629
15630
15631
15632
15633
15634
15635
15636
15637
15638
15639
15640
15641
15642
15643
15644
15645
15646
15647
15648
15649
15650
15651
15652
15653
15654
15655
15656
15657
15658
15659
15660
15661
15662
15663
15664
15665
15666
15667
1566

16867
16868
16869
16870
16871
16872
16873
16874
16875
16876
16877
16878
16879
16880
16881
16882
16883
16884
16885
16886
16887
16888
16889
16890
16891
16892
16893
16894
16895
16896
16897
16898
16899
16900
16901
16902
16903
16904
16905
16906
16907
16908
16909
16910
16911
16912
16913
16914
16915
16916
16917
16918
16919
16920
16921
16922
16923
16924
16925
16926
16927
16928
16929
16930
16931
16932
16933
16934
16935
16936
16937
16938
16939
16940
16941
16942
16943
16944
16945
16946
16947
16948
16949
16950
16951
16952
16953
16954
16955
16956
16957
16958
16959
16960
16961
16962
16963
16964
16965
16966
16967
16968
16969
16970
16971
16972
16973
16974
16975
16976
16977
16978
16979
16980
16981
16982
16983
16984
16985
16986
16987
16988
16989
16990
16991
16992
16993
16994
16995
16996
16997
16998
16999
17000
17001
17002
17003
17004
17005
17006
17007
17008
17009
17010
17011
17012
17013
17014
17015
17016
17017
17018
17019
17020
17021
17022
17023
17024
17025
17026
17027
17028
17029
17030
17031
17032
1703

In [53]:
#test = left.filter(['feature_id','FinancialY','avg_score_lane','Prev_RCI'])

In [54]:
#Add the previous RCI score to the training set ***Separate per lane as well
right['Prev_RCI']= np.nan

for i in range(0,len(right)):
    feat = right.iloc[i,0]
    year = right.iloc[i,1]
    
    print(i)
    
    for t in range(0,10):
        year-=1
        ind = right.loc[(right["feature_id"] == feat) & 
                                (right["FinancialY"] == year)].index
        if len(ind) > 0:
            right.iloc[i,-1] = right.iloc[ind,3].sum()
            break
        
        else:
            right.iloc[i,-1] = right.iloc[i,3].sum()          
            continue

C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059


3498
3499
3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697


5136
5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335


6774
6775
6776
6777
6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973


8412
8413
8414
8415
8416
8417
8418
8419
8420
8421
8422
8423
8424
8425
8426
8427
8428
8429
8430
8431
8432
8433
8434
8435
8436
8437
8438
8439
8440
8441
8442
8443
8444
8445
8446
8447
8448
8449
8450
8451
8452
8453
8454
8455
8456
8457
8458
8459
8460
8461
8462
8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611


In [55]:
ev_set = pd.concat([left, right], ignore_index=True, sort = False)
ev_set.reset_index(drop= True,inplace = True)

In [56]:
ev_set.tail(500)

,feature_id,FinancialY,lane,avg_score_lane,sub_obs_val_max,sub_obs_val_std,Carriageway Scheme,Drainage,Drainage CAT 2,Localised Patching,Long Term Structural Maintenance,Major Patching,Microasphalt,Reconstruction,Recycling,Super Cat 2,Surface Dressing,Surface Inlay,Surface Overlay,Surface Preservative,Thin Surfacing,Years_since,CCD1,CCD3,CCD4,CFL,CFS1,CKD2,CKD3,CMD3,CMUD,CPH2,CPH3,CPH4,CPH7,CSH1,DCF1,DCF2,DCF3,DPD1,DPD2,GUL,NZU,PDAM,RICE,SFL,SPL,SPLE,SURF,BEDR,Month,hierarchy,distance,road_type_hw,urban_rural,surf_type,width,sub_obs_val_avg,weighted_av,perc85,Prev_RCI
26266,C77/10,2016,RIGHT,65.479167,120.00,30.059469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GSG,WHCK,0.0,SD Secondary Distributor,141.00,Single 2-Lane Carriageway,Urban,BITM,15.730000,61.696538,46.291306,96.2300,47.494375
26267,C77/10,2017,RIGHT,52.630714,187.00,50.525165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GSG,WHCK,0.0,SD Secondary Distributor,141.00,Single 2-Lane Carriageway,Urban,BITM,15.730000,52.630714,57.034205,82.7680,65.479167
26268,C77/115,2010,RIGHT,44.345714,140.00,28.126148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,GSG,WHCK,0.0,SD Secondary Distributor,642.00,Single 2-Lane Carriageway,Rural,BITM,11.890000,44.345714,28.814468,62.0580,44.345714
26269,C77/115,2013,RIGHT,56.644762,194.96,43.552728,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GSG,WHCK,0.0,SD Secondary Distributor,642.00,Single 2-Lane Carriageway,Rural,BITM,11.890000,56.644762,39.078933,99.5010,44.345714
26270,C77/115,2016,RIGHT,6.229032,60.00,11.903523,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GSG,WHCK,0.0,SD Secondary Distributor,642.00,Single 2-Lane Carriageway,Rural,BITM,11.890000,6.229032,7.612033,14.6590,56.644762
26271,C77/115,2017,RIGHT,2.282097,59.12,8.938834,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GSG,WHCK,0.0,SD Secondary Distributor,642.00,Single 2-Lane Carriageway,Rural,BITM,11.890000,2.282097,2.803120,0.0000,6.229032
26272,C77/120,2010,RIGHT,9.229200,120.00,22.425291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,GSG,LMBE,0.0,SD Secondary Distributor,757.00,Single 2-Lane Carriageway,Urban,BITM,10.970000,9.229200,11.659452,11.3260,9.229200
26273,C77/120,2014,RIGHT,10.277105,123.78,23.005385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GSG,LMBE,0.0,SD Secondary Distributor,757.00,Single 2-Lane Carriageway,Urban,BITM,10.970000,14.400265,17.413334,36.3950,9.229200
26274,C77/120,2016,RIGHT,12.830533,126.60,22.655010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GSG,LMBE,0.0,SD Secondary Distributor,757.00,Single 2-Lane Carriageway,Urban,BITM,10.970000,12.830533,11.468952,24.2230,10.277105
26275,C77/120,2017,RIGHT,7.648108,103.11,17.346205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GSG,LMBE,0.0,SD Secondary Distributor,757.00,Single 2-Lane Carriagew

In [57]:
#Add the traffic set to the base:
tr_set = ev_set.merge(traffic,'left',on=['feature_id'],left_index = True)
tr_set.reset_index(drop= True, inplace = True)

In [58]:
#Add the weather set to the base:
w_set = tr_set.merge(temp,'left',on=['feature_id', 'FinancialY'],left_index = True)
w_set.reset_index(drop= True, inplace = True)

In [59]:
w_set.head(5)

,feature_id,FinancialY,lane,avg_score_lane,sub_obs_val_max,sub_obs_val_std,Carriageway Scheme,Drainage,Drainage CAT 2,Localised Patching,Long Term Structural Maintenance,Major Patching,Microasphalt,Reconstruction,Recycling,Super Cat 2,Surface Dressing,Surface Inlay,Surface Overlay,Surface Preservative,Thin Surfacing,Years_since,CCD1,CCD3,CCD4,CFL,CFS1,CKD2,CKD3,CMD3,CMUD,CPH2,CPH3,CPH4,CPH7,CSH1,DCF1,DCF2,DCF3,DPD1,DPD2,GUL,NZU,PDAM,RICE,SFL,SPL,SPLE,SURF,BEDR,Month,hierarchy,distance,road_type_hw,urban_rural,surf_type,width,sub_obs_val_avg,weighted_av,perc85,Prev_RCI,AADT_Comb,cloudCover__abs_energy,cloudCover__absolute_sum_of_changes,"cloudCover__agg_autocorrelation__f_agg_""mean""__maxlag_40","cloudCover__agg_autocorrelation__f_agg_""median""__maxlag_40","cloudCover__agg_autocorrelation__f_agg_""var""__maxlag_40","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""intercept""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""rvalue""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""slope""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""stderr""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_5__attr_""intercept""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_5__attr_""rvalue""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_5__attr_""slope""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_5__attr_""stderr""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_10__attr_""intercept""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_10__attr_""rvalue""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_10__attr_""slope""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_10__attr_""stderr""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_50__attr_""intercept""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_50__attr_""rvalue""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_50__attr_""slope""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_50__attr_""stderr""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_5__attr_""intercept""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_5__attr_""rvalue""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_5__attr_""slope""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_5__attr_""stderr""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_10__attr_""intercept""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_10__attr_""rvalue""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_10__attr_""slope""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_10__attr_""stderr""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_50__attr_""intercept""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_50__attr_""rvalue""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_50__attr_""slope""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_50__attr_""stderr""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_5__attr_""intercept""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_5__attr_""rvalue""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_5__attr_""slope""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_5__attr_""stderr""","cloudCover__agg_linear_trend__f_agg_""var""__chunk_len_10__attr_""intercept""","cloudCover__agg_linear_trend__f_agg_""var""__chunk_len_10__attr_""rvalue""","cloudCover__agg_linear_trend__f_agg_""var""__chunk_len_10__attr_""slope""","cloudCover__agg_linear_trend__f_agg_""var""__chunk_len_10__attr_""stderr""","cloudCover__agg_linear_trend__f_agg_""var""__chunk_len_50__attr_""inter

In [60]:
#Structure the gritting set
gritting = grit_routes.merge(grit_runs,'left',on=['FinancialY'],left_index = True)
gritting.reset_index(drop= True, inplace = True)

In [61]:
#Add the gritting set to the base:
final_set = w_set.merge(gritting,'left',on=['feature_id', 'FinancialY'],left_index = True)
final_set.reset_index(drop= True, inplace = True)

In [62]:
final_set.head(5)

,feature_id,FinancialY,lane,avg_score_lane,sub_obs_val_max,sub_obs_val_std,Carriageway Scheme,Drainage,Drainage CAT 2,Localised Patching,Long Term Structural Maintenance,Major Patching,Microasphalt,Reconstruction,Recycling,Super Cat 2,Surface Dressing,Surface Inlay,Surface Overlay,Surface Preservative,Thin Surfacing,Years_since,CCD1,CCD3,CCD4,CFL,CFS1,CKD2,CKD3,CMD3,CMUD,CPH2,CPH3,CPH4,CPH7,CSH1,DCF1,DCF2,DCF3,DPD1,DPD2,GUL,NZU,PDAM,RICE,SFL,SPL,SPLE,SURF,BEDR,Month,hierarchy,distance,road_type_hw,urban_rural,surf_type,width,sub_obs_val_avg,weighted_av,perc85,Prev_RCI,AADT_Comb,cloudCover__abs_energy,cloudCover__absolute_sum_of_changes,"cloudCover__agg_autocorrelation__f_agg_""mean""__maxlag_40","cloudCover__agg_autocorrelation__f_agg_""median""__maxlag_40","cloudCover__agg_autocorrelation__f_agg_""var""__maxlag_40","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""intercept""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""rvalue""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""slope""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""stderr""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_5__attr_""intercept""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_5__attr_""rvalue""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_5__attr_""slope""","cloudCover__agg_linear_trend__f_agg_""max""__chunk_len_5__attr_""stderr""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_10__attr_""intercept""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_10__attr_""rvalue""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_10__attr_""slope""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_10__attr_""stderr""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_50__attr_""intercept""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_50__attr_""rvalue""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_50__attr_""slope""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_50__attr_""stderr""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_5__attr_""intercept""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_5__attr_""rvalue""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_5__attr_""slope""","cloudCover__agg_linear_trend__f_agg_""mean""__chunk_len_5__attr_""stderr""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_10__attr_""intercept""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_10__attr_""rvalue""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_10__attr_""slope""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_10__attr_""stderr""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_50__attr_""intercept""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_50__attr_""rvalue""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_50__attr_""slope""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_50__attr_""stderr""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_5__attr_""intercept""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_5__attr_""rvalue""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_5__attr_""slope""","cloudCover__agg_linear_trend__f_agg_""min""__chunk_len_5__attr_""stderr""","cloudCover__agg_linear_trend__f_agg_""var""__chunk_len_10__attr_""intercept""","cloudCover__agg_linear_trend__f_agg_""var""__chunk_len_10__attr_""rvalue""","cloudCover__agg_linear_trend__f_agg_""var""__chunk_len_10__attr_""slope""","cloudCover__agg_linear_trend__f_agg_""var""__chunk_len_10__attr_""stderr""","cloudCover__agg_linear_trend__f_agg_""var""__chunk_len_50__attr_""inter

In [63]:
final_set.drop(columns=['sub_obs_val_avg','weighted_av','perc85'], inplace = True)

In [64]:
final_set.to_csv(r'C:\Users\J FernandezGomez\Jupyter Notebooks\7_SecondPhase\RCI_Scanner\train_set_ph2.csv')